In [400]:
import numpy as np

In [416]:
class Tensor:
    def __init__(self,data,_children=(),_op=''):
        self.data = data
        self.grad = np.zeros_like(self.data,dtype=np.float32)
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.shape = data.shape
    
    def __repr__(self):
        return str(self.data.__repr__())
        
    def __add__(self,other):
        out = Tensor(self.data + other.data,(self,other),'+')
        
        def _backward():
            self.grad = np.add(self.grad,out.grad)
            other.grad = np.add(self.grad,out.grad)
        out._backward = _backward
        return out
    
    def __mul__(self,other):
        other = other if isinstance(other, Tensor) else Tensor(np.array(other))
        out = Tensor(np.multiply(self.data,other.data),(self,other),'*')
        
        def _backward():
            self.grad += np.multiply(other.data,out.grad)
            other.grad += np.multiply(self.data,out.grad)
        out._backward = _backward
        return out
    
    def matmul(self,other):
        out = Tensor(self.data@other.data,(self,other),'@')
        
        def _backward():
            self.grad += np.dot(out.grad,other.data.T)
            other.grad += np.dot(self.data.T,out.grad)
        out._backward = _backward
        return out    
    
    def relu(self):
        out = Tensor(np.maximum(self.data,0),(self,),'_/')
        
        def _backward():
            self.grad += (out.data>0) * out.grad
        out._backward = _backward
        return out
    def neg(self):
        return self*(-1)
    
    def __sub__(self,other):
        return self + other.neg()
    
    def pow(self,other):
        assert isinstance(other,int)
        out = Tensor((self.data)**other,(self,),'**')
        
        def _backward():
            self.grad += other*(self.data**(other-1))*out.grad
        out._backward = _backward
        return out
    
    def sum(self,axis=None,keepdims=True):
        out = Tensor(np.sum(self.data,axis=axis,keepdims=keepdims),(self,),'++')
        
        def _backward():
            self.grad += np.ones_like(self.data)*out.grad
        out._backward = _backward
        return out
    
    def tanh(self):
        out = Tensor(np.tanh(self.data),(self,),'S')
        
        def _backward():
            self.grad += (1-(out.data**2))*out.grad
        out._backward = _backward
        return out
        
    def backward(self):
        
        topo = []
        visited = set()
        
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.grad = np.ones_like(self.data,dtype=np.float32)
        for node in reversed(topo):
            node._backward()
    
            
        
        
        

In [417]:
import torch

In [418]:
#########

In [419]:
x1 = np.random.rand(2,4)
w1 = np.random.rand(4,2)

x1p = torch.tensor(x1,requires_grad=True,dtype=torch.float32)
w1p = torch.tensor(w1,requires_grad=True,dtype=torch.float32)

x1T = Tensor(x1)
w1T = Tensor(w1)

hp = x1p.matmul(w1p)
hT = x1T.matmul(w1T)

hp = hp.tanh()
hT = hT.tanh()
#print(hp)
lossp = (hp).mean()
#print(lossp)
LossT = hT.sum(axis=1,keepdims=False)
LossT = LossT.sum(axis=0,keepdims=False)*(1/4)
#print(LossT)



lossp.backward()
LossT.backward()

print(np.allclose(w1p.grad,w1T.grad))
print(np.allclose(x1p.grad,x1T.grad))

True
True


In [420]:
########OUR ENGINE AGREES WITH PYTORCH ,AND IT IS MORE PRECISE BUT ALSO CONSUMES MORE MEMORY ~~ HOW MUCH NUMPY CONSUMES

In [421]:
class Layer:
    def __init__(self,in_features,out_features,bias=False):
        self.w = Tensor(np.random.rand(in_features,out_features)*0.1)
        self.b = Tensor(np.random.rand(out_features)) if bias ==True else None
    
    def __call__(self,x):
        out = x.matmul(self.w)
        if self.b is not None:
            out += self.b
        return out
    def parameters(self):
        if self.b is not None:
            return [self.w.data, self.b.data]
        else:
            return self.w.data


        

In [422]:
import torch
import numpy as np

# Set a random seed for reproducibility
torch.manual_seed(1234)
np.random.seed(1234)

# Define PyTorch layer
lp = torch.nn.Linear(2, 6,bias=False)

# Define custom import torch
import numpy as np

# Set a random seed for reproducibility
torch.manual_seed(1234)
np.random.seed(1234)

# Define PyTorch layer
lp = torch.nn.Linear(2, 6, bias=False)

# Define custom layer
lT = Layer(2, 6, bias=False)

# Generate same weights for both layers
lp.weight.data = torch.tensor(lT.w.data.T, dtype=torch.float32)
# lp.bias.data = torch.tensor(lT.b.data)

# Input tensor
x = np.random.rand(2, 2)
xp = torch.tensor(x, requires_grad=True, dtype=torch.float32)
lt = Tensor(x)

# Compute outputs
p = lp(xp)
t = lT(lt)

# Compare outputs
print(p)
print(t)





tensor([[0.0328, 0.0997, 0.0982, 0.1161, 0.0788, 0.0543],
        [0.0226, 0.0680, 0.0700, 0.0782, 0.0490, 0.0382]],
       grad_fn=<MmBackward0>)
array([[0.03279331, 0.09966842, 0.09820385, 0.11610424, 0.07881016,
        0.05433672],
       [0.02260609, 0.06803438, 0.06997732, 0.07823497, 0.04895923,
        0.03820842]])


In [423]:
##also same LAYER WISE

In [424]:
MLP = [Layer(3,4),Layer(4,4),Layer(4,1)]
MLPparams = [l.parameters() for l in MLP ]
MLPparams

[array([[0.05030832, 0.00137684, 0.07728266, 0.08826412],
        [0.0364886 , 0.06153962, 0.00753812, 0.0368824 ],
        [0.09331401, 0.06513781, 0.03972026, 0.07887301]]),
 array([[0.03168361, 0.05680987, 0.08691274, 0.04361734],
        [0.08021476, 0.01437668, 0.0704261 , 0.07045813],
        [0.02187921, 0.09248676, 0.04421408, 0.0909316 ],
        [0.00598092, 0.01842871, 0.00473553, 0.06748809]]),
 array([[0.05946248],
        [0.05333102],
        [0.00433241],
        [0.05614331]])]

In [425]:
for layer in MLP:
    print(layer.w.shape)


(3, 4)
(4, 4)
(4, 1)


In [426]:
x = Tensor(np.random.rand(4,3))


In [427]:
MLP[0].w.shape,x.shape#,MLP[0](x)

((3, 4), (4, 3))

In [428]:
x = Tensor(np.array([2.0, 3.0, -1.0]))

for l in MLP:
    x = l(x)
x

array([0.00475852])

In [429]:
xs = Tensor(np.array([
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]))
ys = Tensor(np.array([1.0, -1.0, -1.0, 1.0]))

In [431]:
""

''

In [188]:
a = np.zeros((2,2))
b = np.ones((2,2))
a

array([[0., 0.],
       [0., 0.]])

In [157]:
a = Tensor(a)
b = Tensor(b)

c = b.neg()
c

array([[-1., -1.],
       [-1., -1.]])